## Neural Networks

Deep Learning is based on artificial neural networks which have been around in some form since the late 1950s. The networks are built from individual parts approximating neurons, typically called units or simply "neurons." Each unit has some number of weighted inputs. These weighted inputs are summed together (a linear combination) then passed through an activation function to get the unit's output.

<img src="assets/simple_neuron.png" width=400px>

Mathematically this looks like: 

$$
\begin{align}
y &= f(w_1 x_1 + w_2 x_2 + b) \\
y &= f\left(\sum_i w_i x_i +b \right)
\end{align}
$$

With vectors this is the dot/inner product of two vectors:

$$
h = \begin{bmatrix}
x_1 \, x_2 \cdots  x_n
\end{bmatrix}
\cdot 
\begin{bmatrix}
           w_1 \\
           w_2 \\
           \vdots \\
           w_n
\end{bmatrix}
$$

## Tensors

It turns out neural network computations are just a bunch of linear algebra operations on *tensors*, a generalization of matrices. A vector is a 1-dimensional tensor, a matrix is a 2-dimensional tensor, an array with three indices is a 3-dimensional tensor (RGB color images for example). The fundamental data structure for neural networks are tensors and PyTorch (as well as pretty much every other deep learning framework) is built around tensors.

<img src="assets/tensor_examples.svg" width=600px>

With the basics covered, it's time to explore how we can use PyTorch to build a simple neural network.

In [1]:
# First, import PyTorch
import torch

In [2]:
def activation(x):
    """ Sigmoid activation function 
    
        Arguments
        ---------
        x: torch.Tensor
    """
    return 1/(1+torch.exp(-x))

In [3]:
### Generate some data
torch.manual_seed(7) # Set the random seed so things are predictable

# Features are 3 random normal variables
features = torch.randn((1, 5))
# True weights for our data, random normal variables again
weights = torch.randn_like(features)
# and a true bias term
bias = torch.randn((1, 1))

In [4]:
activation((features * weights).sum() + bias)

tensor([[0.1595]])

In [5]:
## Calculate the output of this network using matrix multiplication
weights = weights.view(5,1)
activation(torch.mm(features,weights) + bias)

tensor([[0.1595]])

### Stack them up!

That's how you can calculate the output for a single neuron. The real power of this algorithm happens when you start stacking these individual units into layers and stacks of layers, into a network of neurons. The output of one layer of neurons becomes the input for the next layer. With multiple input units and output units, we now need to express the weights as a matrix.

<img src='assets/multilayer_diagram_weights.png' width=450px>

The first layer shown on the bottom here are the inputs, understandably called the **input layer**. The middle layer is called the **hidden layer**, and the final layer (on the right) is the **output layer**. We can express this network mathematically with matrices again and use matrix multiplication to get linear combinations for each unit in one operation. For example, the hidden layer ($h_1$ and $h_2$ here) can be calculated 

$$
\vec{h} = [h_1 \, h_2] = 
\begin{bmatrix}
x_1 \, x_2 \cdots \, x_n
\end{bmatrix}
\cdot 
\begin{bmatrix}
           w_{11} & w_{12} \\
           w_{21} &w_{22} \\
           \vdots &\vdots \\
           w_{n1} &w_{n2}
\end{bmatrix}
$$

The output for this small network is found by treating the hidden layer as inputs for the output unit. The network output is expressed simply

$$
y =  f_2 \! \left(\, f_1 \! \left(\vec{x} \, \mathbf{W_1}\right) \mathbf{W_2} \right)
$$

In [6]:
### Generate some data
torch.manual_seed(7) # Set the random seed so things are predictable

# Features are 3 random normal variables
features = torch.randn((1, 3))

# Define the size of each layer in our network
n_input = features.shape[1]     # Number of input units, must match number of input features
n_hidden = 2                    # Number of hidden units 
n_output = 1                    # Number of output units

# Weights for inputs to hidden layer
W1 = torch.randn(n_input, n_hidden)
# Weights for hidden layer to output layer
W2 = torch.randn(n_hidden, n_output)

# and bias terms for hidden and output layers
B1 = torch.randn((1, n_hidden))
B2 = torch.randn((1, n_output))

In [7]:
## Your solution here
l1 = activation(torch.mm(features,W1) + B1)
activation(torch.mm(l1,W2)+B2)

tensor([[0.3171]])

## Numpy to Torch and back

Special bonus section! PyTorch has a great feature for converting between Numpy arrays and Torch tensors. To create a tensor from a Numpy array, use `torch.from_numpy()`. To convert a tensor to a Numpy array, use the `.numpy()` method.

In [8]:
import numpy as np
np.set_printoptions(precision=8)
a = np.random.rand(4,3)
a

array([[0.83207082, 0.98393234, 0.3379819 ],
       [0.13931795, 0.13878253, 0.40786441],
       [0.6300598 , 0.26411612, 0.64072385],
       [0.54986631, 0.85552863, 0.52448797]])

In [9]:
torch.set_printoptions(precision=8)
b = torch.from_numpy(a)
b

tensor([[0.83207082, 0.98393234, 0.33798190],
        [0.13931795, 0.13878253, 0.40786441],
        [0.63005980, 0.26411612, 0.64072385],
        [0.54986631, 0.85552863, 0.52448797]], dtype=torch.float64)

In [10]:
b.numpy()

array([[0.83207082, 0.98393234, 0.3379819 ],
       [0.13931795, 0.13878253, 0.40786441],
       [0.6300598 , 0.26411612, 0.64072385],
       [0.54986631, 0.85552863, 0.52448797]])

The memory is shared between the Numpy array and Torch tensor, so if you change the values in-place of one object, the other will change as well.

In [11]:
# Multiply PyTorch Tensor by 2, in place
b.mul_(2)

tensor([[1.66414164, 1.96786468, 0.67596381],
        [0.27863590, 0.27756506, 0.81572882],
        [1.26011959, 0.52823223, 1.28144770],
        [1.09973262, 1.71105725, 1.04897593]], dtype=torch.float64)

In [12]:
# Numpy array matches new values from Tensor
a

array([[1.66414164, 1.96786468, 0.67596381],
       [0.2786359 , 0.27756506, 0.81572882],
       [1.26011959, 0.52823223, 1.2814477 ],
       [1.09973262, 1.71105725, 1.04897593]])

In [13]:
# inplace operation done on tensors can also change the origin numpy array